In [ ]:
## Predicting Survival on the Titanic

### History
Perhaps one of the most infamous shipwrecks in history, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 people on board. Interestingly, by analysing the probability of survival based on few attributes like gender, age, and social status, we can make very accurate predictions on which passengers would survive. Some groups of people were more likely to survive than others, such as women, children, and the upper-class. Therefore, we can learn about the society priorities and privileges at the time.

### Assignment:

Build a Machine Learning Pipeline, to engineer the features in the data set and predict who is more likely to Survive the catastrophe.

Follow the Jupyter notebook below, and complete the missing bits of code, to achieve each one of the pipeline steps.

In [ ]:
import re

# to handle datasets
import pandas as pd
import numpy as np

# for visualization
import matplotlib.pyplot as plt

# to divide train and test set
from sklearn.model_selection import train_test_split

# feature scaling
from sklearn.preprocessing import StandardScaler

# feature scaling
#from sklearn.preprocessing import MinMaxScaler

# to build the models
from sklearn.linear_model import LogisticRegression

# to evaluate the models
from sklearn.metrics import accuracy_score, roc_auc_score

# to persist the model and the scaler
import joblib

# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)

# to build the model
from sklearn.linear_model import Lasso

# to evaluate the model
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)

In [ ]:
## Prepare the data set

In [ ]:
# load the data - it is available open source and online

data = pd.read_csv('https://www.openml.org/data/get_csv/16826755/phpMYEkMl')

# display data
data.head()

data.shape

In [ ]:
# retain only the first cabin if more than
# 1 are available per passenger

def get_first_cabin(row):
    try:
        return row.split()[0]
    except:
        return np.nan
    
data['cabin'] = data['cabin'].apply(get_first_cabin)

In [ ]:
# extracts the title (Mr, Ms, etc) from the name variable

def get_title(passenger):
    line = passenger
    if re.search('Mrs', line):
        return 'Mrs'
    elif re.search('Mr', line):
        return 'Mr'
    elif re.search('Miss', line):
        return 'Miss'
    elif re.search('Master', line):
        return 'Master'
    else:
        return 'Other'
    
data['title'] = data['name'].apply(get_title)

In [ ]:
# cast numerical variables as floats

data['fare'] = data['fare'].astype('float')
data['age'] = data['age'].astype('float')

In [ ]:
# drop unnecessary variables

data.drop(labels=['name','ticket', 'boat', 'body','home.dest'], axis=1, inplace=True)

# display data
data.head()


In [ ]:
data.shape

In [ ]:
# save the data set

data.to_csv('titanic.csv', index=False)

In [ ]:
## Data Exploration

### Find numerical and categorical variables

In [ ]:
target = 'survived'

In [ ]:
#vars_num = # fill your code here
vars_num = [var for var in data.columns if data[var].dtypes != 'O']
# visualise the numerical variables
data[vars_num].head()

#vars_cat = # fill your code here
vars_cat = [var for var in data.columns if data[var].dtypes == 'O']

print('Number of numerical variables: {}'.format(len(vars_num)))
print('Number of categorical variables: {}'.format(len(vars_cat)))

In [ ]:
### Find missing values in variables

In [ ]:
data[vars_num].head()

In [ ]:
data[vars_cat].head()

In [ ]:
# first in numerical variables

# make a list of the variables that contain missing values
vars_with_na_1 = [var for var in vars_num if data[var].isnull().sum() > 0]

# determine percentage of missing values
data[vars_with_na_1].isnull().mean()

In [ ]:
# now in categorical variables

# make a list of the variables that contain missing values
vars_with_na_2 = [var for var in vars_cat if data[var].isnull().sum() > 0]

# determine percentage of missing values
data[vars_with_na_2].isnull().mean()

In [ ]:
### Determine cardinality of categorical variables

In [ ]:
data[vars_cat].nunique()

In [ ]:
### Determine the distribution of numerical variables

In [ ]:
data[vars_num].nunique()

In [ ]:
def analyse_continuous(df, var):
    df = df.copy()
    df[var].hist(bins=15)
    plt.ylabel('No of Passengers')
    plt.xlabel(var)
    plt.title(var)
    plt.show()


for var in vars_num:
    analyse_continuous(data, var)

In [ ]:
## Separate data into train and test

Use the code below for reproducibility. Don't change it.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('survived', axis=1),  # predictors
    data['survived'],  # target
    test_size=0.2,  # percentage of obs in test set
    random_state=0)  # seed to ensure reproducibility

X_train.shape, X_test.shape

In [ ]:
## Feature Engineering

### Extract only the letter (and drop the number) from the variable Cabin

In [ ]:
cabins = data['cabin']
cabins.shape

In [ ]:
vars_first_cabin_char = []


In [ ]:
for x in range(0, len(cabins)):
    _cabin = cabins[x]
    
    if pd.isnull(_cabin):
        vars_first_cabin_char.append('N')
    else:    
        letter = _cabin[0]
        vars_first_cabin_char.append(letter)
      
       
print(vars_first_cabin_char)


In [ ]:
### Fill in Missing data in numerical variables:

- Add a binary missing indicator
- Fill NA in original variable with the median

In [ ]:
def analyse_na_value(df, var):

    df = df.copy()

    # let's make a variable that indicates 1 if the observation was missing or zero otherwise
    df[var] = np.where(df[var].isnull(), 1, 0)

    # let's compare the median survived in the observations where data is missing
    # vs the observations where a value is available

    df.groupby(var).median().plot.bar()

    plt.title(var)
    plt.show()


# let's run the function on each variable with missing data
for var in vars_with_na_1:
    analyse_na_value(data, var)

In [ ]:
# replace engineer missing values with mod value

for var in vars_with_na_1:

    # calculate the mode using the train set
    mode_val = X_train[var].mode()[0]

    # add binary missing indicator (in train and test)
    X_train[var+'_na_1'] = np.where(X_train[var].isnull(), 1, 0)
    X_test[var+'_na_1'] = np.where(X_test[var].isnull(), 1, 0)

    # replace missing values by the mode
    # (in train and test)
    X_train[var] = X_train[var].fillna(mode_val)
    X_test[var] = X_test[var].fillna(mode_val)



In [ ]:
# check that we have no more missing values in the engineered variables
X_train[vars_with_na_1].isnull().sum()

In [ ]:
### Replace Missing data in categorical variables with the string **Missing**

In [ ]:
#### Rare labels:

#Let's go ahead and investigate now if there are labels that are present only in a small number of houses:

def analyse_rare_labels(df, var, rare_perc):
    df = df.copy()

    # determine the % of observations per category
    tmp = df.groupby(var)['survived'].count() / len(df)

    # return categories that are rare
    return tmp[tmp < rare_perc]

# print categories that are present in less than
# 1 % of the observations


for var in vars_cat:
    print(analyse_rare_labels(data, var, 0.01))
    print()

In [ ]:
def analyse_discrete(df, var):
    df = df.copy()
    df.groupby(var).median().plot.bar()
    plt.title(var)
    plt.ylabel('')
    plt.show()

for var in vars_cat:
    # we can re-use the function to determine median
    # sale price, that we created for discrete variables

    analyse_discrete(data, var)

In [ ]:
# replace missing values with new label: "Missing"

X_train[vars_with_na_2] = X_train[vars_with_na_2].fillna('Missing')
X_test[vars_with_na_2] = X_test[vars_with_na_2].fillna('Missing')

In [ ]:
### Remove rare labels in categorical variables

- remove labels present in less than 5 % of the passengers

In [ ]:
def find_frequent_labels(df, var, rare_perc):
    
    # function finds the labels that are shared by more than
    # a certain % of the houses in the dataset

    df = df.copy()

    tmp = df.groupby(var).count() / len(df)

    return tmp[tmp > rare_perc].index


for var in vars_cat:
    
    # find the frequent categories
    frequent_ls = find_frequent_labels(X_train, var, 0.005)
    
    # replace rare categories by the string "Rare"
    X_train[var] = np.where(X_train[var].isin(
        frequent_ls), X_train[var], 'Rare')
    
    X_test[var] = np.where(X_test[var].isin(
        frequent_ls), X_test[var], 'Rare')

In [ ]:
# check that we have no missing information in the engineered variables
X_train[vars_with_na_1].isnull().sum()

In [ ]:
# check that test set does not contain null values in the engineered variables
[var for var in vars_with_na_2 if X_test[var].isnull().sum() > 0]

In [ ]:
### Perform one hot encoding of categorical variables into k-1 binary variables

- k-1, means that if the variable contains 9 different categories, we create 8 different binary variables
- Remember to drop the original categorical variable (the one with the strings) after the encoding

In [ ]:
# this function will assign discrete values to the strings of the variables,

def replace_categories(train, test, var, target):

    # order the categories in a variable from that with the lowest
    # survived, to that with the highest
    ordered_labels = train.groupby([var])[target].mean().sort_values().index

    # create a dictionary of ordered categories to integer values
    ordinal_label = {k: i for i, k in enumerate(ordered_labels, 0)}

    # use the dictionary to replace the categorical strings by integers
    train[var] = train[var].map(ordinal_label)
    test[var] = test[var].map(ordinal_label)

In [ ]:
for var in vars_cat:
    replace_categories(X_train, X_test, var, 'age')

In [ ]:
# check absence of na in the train set
[var for var in X_train.columns if X_train[var].isnull().sum() > 0]

In [ ]:
# check absence of na in the test set
[var for var in X_test.columns if X_test[var].isnull().sum() > 0]

In [ ]:
### Scale the variables

- Use the standard scaler from Scikit-learn

In [ ]:
# capture all variables in a list
# except the target and the ID

train_vars = [var for var in X_train.columns if var not in ['survived']]

# count number of variables
len(train_vars)

In [ ]:
# create scaler
scaler = StandardScaler()

#  fit  the scaler to the train set
scaler.fit(X_train[train_vars]) 

# transform the train and test set
X_train[train_vars] = scaler.transform(X_train[train_vars])

X_test[train_vars] = scaler.transform(X_test[train_vars])

In [ ]:
## Train the Logistic Regression model

- Set the regularization parameter to 0.0005
- Set the seed to 0

In [ ]:
# set up the model
# remember to set the random_state / seed

lin_model = Lasso(alpha=0.0005, random_state=0)

# train the model

lin_model.fit(X_train, y_train)

In [ ]:
## Make predictions and evaluate model performance

Determine:
- roc-auc
- accuracy

**Important, remember that to determine the accuracy, you need the outcome 0, 1, referring to survived or not. But to determine the roc-auc you need the probability of survival.**

In [ ]:
# evaluate the model:
# ====================

# remember that we log transformed the output (SalePrice)
# in our feature engineering notebook (step 2).

# In order to get the true performance of the Lasso
# we need to transform both the target and the predictions
# back to the original house prices values.

# We will evaluate performance using the mean squared error and
# the root of the mean squared error and r2

# make predictions for train set
pred = lin_model.predict(X_train)

# determine mse and rmse
print('train mse: {}'.format(int(
    mean_squared_error(np.exp(y_train), np.exp(pred)))))
print('train rmse: {}'.format(int(
    sqrt(mean_squared_error(np.exp(y_train), np.exp(pred))))))
print('train r2: {}'.format(
    r2_score(np.exp(y_train), np.exp(pred))))
print()

# make predictions for test set
pred = lin_model.predict(X_test)

# determine mse and rmse
print('test mse: {}'.format(int(
    mean_squared_error(np.exp(y_test), np.exp(pred)))))
print('test rmse: {}'.format(int(
    sqrt(mean_squared_error(np.exp(y_test), np.exp(pred))))))
print('test r2: {}'.format(
    r2_score(np.exp(y_test), np.exp(pred))))
print()

print('Average survivability: ', int(np.exp(y_train).median()))

In [ ]:
That's it! Well done

**Keep this code safe, as we will use this notebook later on, to build production code, in our next assignement!!**